In [27]:
import os
from dotenv import load_dotenv
from yandex_chain import YandexLLM, YandexEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.pdf import UnstructuredPDFLoader, PDFMinerLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from tqdm import tqdm


load_dotenv()

True

In [9]:
embeddings = YandexEmbeddings(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))
# text_splitter = CharacterTextSplitter(chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)


for path in os.listdir('../files/txts'):
    txt_path = '../files/txts/'+path
    loader = TextLoader(txt_path)
    document = loader.load()
    doc = text_splitter.split_documents(document)
    db = FAISS.from_documents(doc, embeddings)
    print(db.index.ntotal)

73
52
68
68
161
127
174
197
218
182
324


In [10]:
db.save_local('../files', index_name='finance_docs')

In [12]:
query = "Что по гособлигациям?"
print(f"+ Запрос: {query}")

print(" + Получение релевантных документов")
retriever = db.as_retriever()
result = retriever.get_relevant_documents(query)
for x in result:
    print(f"{x}\n------\n")

+ Запрос: Что по гособлигациям?
 + Получение релевантных документов
page_content='договорные  условия  финансового  актива  обуславливают  получение  в  указанные  даты \nденежных  потоков,  являющихся  исключительно  платежами в  счет  основной суммы долга и \nпроцентов на непогашенную часть основной суммы долга. \n\uf0b7 \n \nПри  первоначальном  признании  Группа  может  принять  решение,  без  права  его  последующей \nотмены, представлять в  составе  прочего  совокупного дохода  изменения  справедливой стоимости \nинвестиции в долевой инструмент, не предназначенный для торговли. Такое решение принимается \nпо каждому инструменту в отдельности. \n \n \n22' metadata={'source': '../files/txts/Консолидированная финансовая отчетность за год закончившийся 31 декабря 2022 г. (в рублях)_.txt'}
------

page_content='учитываются  в  российских  депозитариях,  произошла  автоматически.  Конвертация  депозитарных \nрасписок,  права  на  которые  учитываются  в  иностранных  депозитариях,  был

/home/jupyter/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [14]:
template = """Ответь на вопрос, основываясь только на текст ниже:
{context}

Вопрос: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model = YandexLLM(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

print(" + Запуск LLM Chain")
res = chain.invoke(query)
print(f" + Ответ: {res}")

 + Запуск LLM Chain
 + Ответ: Исходя из предоставленного текста, можно сделать вывод, что по состоянию на 31 декабря 2022 года **по гособлигациям не проводились торги**, но они были возобновлены в конце марта того же года. При этом были **действительны ограничения на сделки с ценными бумагами,** которые совершались нерезидентами, так как торги проходили на Московской бирже. 

Вероятно, вы имеете в виду что-то ещё — пожалуйста, уточните ваш вопрос.


In [21]:
! ls ../files/pdfs/all_docs

27.04.2023 АЗ ГД МСФО за 2022г_опт.pdf
O1 Properties Finance FS 2022 signed.pdf
Summary_MICEX-RTS_FS_4Q2022_RUS.pdf
x5_q2_2022_financial_results_rus.pdf
АЗ+КФО Абрау-Дюрсо_2022 (3).pdf
Консолидированная финансовая отчетность за год закончившийся 31 декабря 2022 г. (в рублях)_.pdf
МСФО отчетность 2022_УК Первая_2022.pdf
Отчетность МСФО 2022.pdf
Отчетность и аудиторское заключение за 2022 год Мэйл-Copy1.Ру Финанс.pdf
Отчетность и аудиторское заключение за 2022 год Мэйл.Ру Финанс.pdf
Финансовая отчетность по МСФО (12 месяцев 2022 года).pdf


In [30]:
# Не успею создать еще один варинат db

# def create_embeddings_with_pdf(folder_path: str, index_name: str) -> FAISS:
#     embeddings = YandexEmbeddings(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

#     for path in tqdm(os.listdir('../files/pdfs/all_docs')):
#         pdf_path = '../files/pdfs/all_docs/'+path
#         loader = PDFMinerLoader(pdf_path)
#         document = loader.load()
#         doc = text_splitter.split_documents(document)
#         db = FAISS.from_documents(doc, embedding=embeddings)  # .from_documents(doc, embeddings)
    
#     db.save_local(folder_path, index_name)

#     return db

# db_pdf = create_embeddings_with_pdf('../files/dbs/db1_pdf', index_name='finance_docs')

In [36]:
import os
from tqdm import tqdm
from dotenv import load_dotenv
from yandex_chain import YandexLLM, YandexEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.pdf import PDFMinerLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

load_dotenv()


def create_embeddings_with_text(folder_path: str, index_name: str) -> FAISS:
    embeddings = YandexEmbeddings(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))
    # text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

    for path in tqdm(os.listdir('../files/txts')):
        txt_path = '../files/txts/'+path
        loader = TextLoader(txt_path)
        document = loader.load()
        doc = text_splitter.split_documents(document)
        db = FAISS.from_documents(doc, embeddings)
    
    db.save_local(folder_path, index_name)

    return db


def create_embeddings_with_pdf(folder_path: str, index_name: str) -> FAISS:
    embeddings = YandexEmbeddings(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

    for path in tqdm(os.listdir('../files/pdfs/all_docs')):
        pdf_path = '../files/pdfs/all_docs/'+path
        loader = PDFMinerLoader(pdf_path)
        document = loader.load()
        doc = text_splitter.split_documents(document)
        db = FAISS.from_documents(doc, embedding=embeddings)  # .from_documents(doc, embeddings)
    
    db.save_local(folder_path, index_name)

    return db


def load_embeddings(folder_path: str, index_name: str) -> FAISS:
    embeddings = YandexEmbeddings(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))
    db = FAISS.load_local(folder_path=folder_path, embeddings=embeddings, index_name=index_name, allow_dangerous_deserialization=True)
    return db


def rag_system(query: str, db_folder_path: str, db_index_name: str):
    # print(" + Индексируем документы")
    # db = create_embeddings_with_text('../files/dbs/db1_text', index_name='finance_docs') Создаем базу данных, если ее нет
    # db = create_embeddings_with_pdf('../files/dbs/db1_pdf', index_name='finance_docs') Второй варинант создания без custom pdf_parser
    db =  load_embeddings(folder_path=db_folder_path, index_name=db_index_name)
    retriever = db.as_retriever()

    # print(f"+ Запрос: {query}")
    # print(" + Получение релевантных документов")
    retriever = db.as_retriever()
    # docs = retriever.invoke(query) На будущее
    # print(docs[0].page_content) по этому print можно будет определить в каком месте плохо сформирован текст pdf парсером

    template = """Ответь на вопрос, основываясь только на текст ниже. Плюс сначала пиши название текста и через 2 строчки ответ.
    Еще учитывай, что ты отлично разбираешься в финансовых отчетах и бухгалтерском учете в целом.
    Также помни ты получаешь данные из нескольких документов в общей базе данных:
    {context}

    Вопрос: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)
    model = YandexLLM(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))

    chain = (
        {"context": retriever, "question": RunnablePassthrough()} 
        | prompt 
        | model 
        | StrOutputParser()
    )

    # print(" + Запуск LLM Chain")
    res = chain.invoke(query)
    print(f" + Ответ: {res}")




def main():
    while True:
        query = input('Введите ваш запрос (или q для выхода): ')
        if query == 'q': break
        rag_system(query=query, db_folder_path='../files/dbs/db1_text', db_index_name='finance_docs')
        print('\n\n' + 100*'=' + '\n\n')


if __name__ == "__main__":
    main()

Введите ваш запрос (или q для выхода):  Когда прекращается признание финансовых обязательств


 + Ответ: Прекращение признания финансовых обязательств происходит в случае исполнения, отмены или истечения срока обязательств.






Введите ваш запрос (или q для выхода):  q


In [44]:
test_query = "Когда прекращается признание финансовых обязательств"
docs_and_scores = db.similarity_search_with_score(test_query)
print(docs_and_scores[-2])  # можно еще подумать, как увеличить качество, потому что по score лучше -1 документ, а ответом будет -2 документ

(Document(page_content='4.  ОСНОВНЫЕ ПОЛОЖЕНИЯ УЧЕТНОЙ ПОЛИТИКИ (ПРОДОЛЖЕНИЕ) \nПрекращение признания финансовых активов \n \nГруппа списывает финансовый актив с учета только в случае прекращения прав на денежные потоки \nпо соответствующему договору или в случае передачи финансового актива и всех соответствующих \nосновных рисков и выгод от владения активом другой организации. Если Группа не передает, но в \nто  же  время  не  сохраняет  основные  риски  и  выгоды  от  владения  активом  и  продолжает \nконтролировать  переданный  актив,  то  она  продолжает  отражать  свою  долю  в  данном  активе  и \nсвязанном  с  ним  обязательстве  в  сумме,  предполагаемой  к  уплате.  Если  Группа  сохраняет  все \nосновные  риски  и  выгоды  от  владения  переданным  финансовым  активом,  она  продолжает \nучитывать данный финансовый актив, а также отражает обеспеченный заем в сумме полученных \nсредств. \n \nФинансовые обязательства', metadata={'source': '../files/txts/Консолидированная финан